In [21]:
import pandas as pd
from fastapi import FastAPI, HTTPException
from datetime import date
import unicodedata

In [55]:
df = pd.read_csv('C:/Users/house/Desktop/joined_dataset.csv')
df_dir = pd.read_csv('C:/Users/house/Desktop/directores.csv')
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [56]:
df_dir.head(5)

,name
0,John Lasseter
1,Joe Johnston
2,Howard Deutch
3,Forest Whitaker
4,Charles Shyer


In [14]:
def cantidad_filmaciones_dia2(dia:str):
    dia = dia.lower().strip()
    dias_semana = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
    
    # Validamos si el día es correcto
    if dia not in dias_semana:
        raise HTTPException(status_code=404, detail="Día no válido")
    
    # Mapeamos cada día de la semana a su valor numérico (0 = lunes, ..., 6 = domingo)
    dia_numero = {
        'lunes': 0,
        'martes': 1,
        'miercoles': 2,
        'jueves': 3,
        'viernes': 4,
        'sabado': 5,
        'domingo': 6
    }
    
    # Convertimos las fechas a números de día de la semana y contamos ocurrencias
    matriz = df['release_date'].dt.dayofweek.value_counts()
    cantidad = matriz.get(dia_numero[dia], 0)
    
    # Devolvemos el mensaje con el valor de cantidad
    return {"mensaje": f"{cantidad} cantidad de películas fueron estrenadas en los días {dia}"}

In [42]:
def cantidad_filmaciones_mes(mes:str):
    mes = mes.lower().strip()
    meses_anio = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 
                'junio', 'julio', 'agosto', 'septiembre', 'octubre', 
                'noviembre', 'diciembre']
    
    # Validamos si el día es correcto
    if mes not in meses_anio:
        raise HTTPException(status_code=404, detail="Mes no válido")
    
    # Mapeamos cada mes del año a su valor numérico (0 = enero, ..., 12 = diciembre)
    mes_numero = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
                    'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
                    'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    # Convertimos los meses a números de mes del año y contamos ocurrencias
    matriz = df['release_date'].dt.month.value_counts()
    cantidad = matriz.get(mes_numero[mes], 0)
    
    print(f"{cantidad} cantidad de películas fueron estrenadas en el mes de {mes}")

In [43]:
cantidad_filmaciones_mes('diciembre')

1065 cantidad de películas fueron estrenadas en el mes de diciembre


In [53]:
def votos_titulo(titulo_de_la_filmacion:str):

    titulo_de_la_filmacion = titulo_de_la_filmacion.title().strip() #Normalizamos el string
    
    row_number = df.index.get_loc(df[df['title'] == titulo_de_la_filmacion].index[0])
    anio = int(df['year'][row_number])    
    promedio = float(df['vote_average'][row_number])
    val = int(df['vote_count'][row_number])
    if int(df['vote_count'][row_number]) >= 2000:
        print(f'La película {titulo_de_la_filmacion} fue estrenada en el año {anio}. La misma cuenta con un total de {val} valoraciones, con un promedio de {promedio}')
    else:
        print('La película tiene menos de 2000 valoraciones')

In [54]:
votos_titulo('jumanji')

La película Jumanji fue estrenada en el año 1995. La misma cuenta con un total de 2413 valoraciones, con un promedio de 6.9


In [ ]:
def get_actor(nombre_actor:str):    
    nombre_actor = nombre_actor.title().strip()

    
    try:
        actores = df[df['cast'] == nombre_actor].iloc[0]
    except IndexError:
        raise HTTPException(status_code=404, detail="El actor no fue encontrado")
    
    director = df_dir.query('name == @nombre_actor')
    existencia = director['name'].count()

    if existencia > 0:
        print('El actor ha trabajado como director')

    else:
        matriz = df.query('cast == @nombre_actor')  
        peliculas = matriz['id'].count()
        retorno = matriz['return'].sum()
        promedio = matriz['return'].mean()

        print(f'El actor {nombre_actor} ha participado de {peliculas} cantidad de filmaciones, el mismo ha conseguido un retorno de {retorno} con un promedio de {promedio} por filmación')
    

In [63]:
get_actor('robin williams')

El actor Robin Williams ha participado de 32 cantidad de filmaciones, el mismo ha conseguido un retorno de 50.98922394518207 con un promedio de 1.5934132482869396 por filmación
